<a href="https://colab.research.google.com/github/Ayush6459/DLCVNLP_practice/blob/master/CNN/Notebooks/Resnet50_pertained_on_cifar100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the libraries

import tensorflow as tf

from tensorflow.keras.layers import Reshape, Dropout, Dense,Multiply, Dot, Concatenate,Embedding, Conv2D, Input

In [ ]:

import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.datasets import cifar100
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import UpSampling2D, Flatten

In [ ]:
con_base =ResNet50(weights='imagenet',include_top=False,input_shape=(128,128,3))
con_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [ ]:
model = models.Sequential()
model.add(layers.UpSampling2D((2,2)))  #32*2 =64  ## UpSampling increase the row and column of the data.Sometimes if we have less data so we can try to increase the data in this way.
model.add(layers.UpSampling2D((2,2)))  #64*2 = 128
#model.add(layers.UpSampling2D((2,2)))  #128*2 = 256
#model.add(tf.keras.layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear"))
model.add(con_base)                             ## conv_base is the inception network.We are keeping it here.

model.add(layers.Flatten())
model.add(layers.Dense(2048, activation='relu' ))  
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1000,activation='relu'))
model.add(layers.Dense(100, activation='softmax')) 

In [ ]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer = RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 513s 600ms/step - loss: 1.6336 - acc: 0.5772 - val_loss: 3.1660 - val_acc: 0.3156
Epoch 2/10
782/782 [==============================] - 467s 597ms/step - loss: 0.5947 - acc: 0.8247 - val_loss: 1.3536 - val_acc: 0.6820
Epoch 3/10
782/782 [==============================] - 467s 597ms/step - loss: 0.2538 - acc: 0.9231 - val_loss: 1.6342 - val_acc: 0.7115
Epoch 4/10
782/782 [==============================] - 466s 597ms/step - loss: 0.1672 - acc: 0.9508 - val_loss: 1.9815 - val_acc: 0.7020
Epoch 5/10
782/782 [==============================] - 467s 597ms/step - loss: 0.1344 - acc: 0.9612 - val_loss: 4.1481 - val_acc: 0.6655
Epoch 6/10
782/782 [==============================] - 467s 597ms/step - loss: 0.1173 - acc: 0.9676 - val_loss: 5.5030 - val_acc: 0.6804
Epoch 7/10
782/782 [==============================] - 466s 596ms/step - loss: 0.1015 - acc: 0.9707 - val_loss: 8.2794 - val_acc: 0.5812
Epoch 8/10
782/782 [============================

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()